## Uavs Collision

Collision of 2 uavs


In [ ]:
import numpy as np

# Custom modules
from helpers import global2local, plot_3d_interactive, kill_processes
from simulators import Simulator, Gazebo, QGC

from plan import Plan, State
from oracle import Oracle


kill_processes()

## Create Plans

In [ ]:
offsets = [(-20, 0, 0, 0),(20, 0, 0, 0)]#
n_vehicles=len(offsets)


homes=np.array([offset[:3] for offset in offsets])
global_paths=[np.array([[-20, 0, 5],[20, 0, 5]]),
              np.array([[20, 0, 5],[-20, 0, 5]])]
local_paths=[global2local(path, home) for path,home in zip(global_paths,homes)]
plans=[Plan.basic(wps=path,wp_margin=0.5,navegation_speed=10) for path in local_paths]

## Visualization Parameters

In [ ]:
waypoints = {f'waypoint_{i+1}': {'pos':global_paths[0],'color':'blue'} for i in range(n_vehicles)}
plot_3d_interactive(waypoints,title='Simulation Markers',expand=[0.2,0.2,0.6],ground=-0.05)

## Choose Simulator

In [ ]:
# simulator = Simulator(name="NONE",offsets=offsets,plans=plans)

# simulator=QGC(offsets=offsets,
#               plans=plans,
#               origin=(-35.3633245,149.1652241,0,0))

simulator=Gazebo(offsets=offsets,
                 plans=plans,
                 world_path='simulators/gazebo/worlds/runway.world',
                 vehicle_models=n_vehicles*['drone'],
                 markers=waypoints)

Launch Simulator

In [ ]:
uavs = simulator.launch()
orac = Oracle(uavs)

## Execute Plan

In [ ]:
while orac.vehs:
    for uav in orac.vehs.copy():
        if uav.plan.state == State.DONE:
            orac.remove(uav)
        else:
            uav.act()
kill_processes()

In [ ]:
plans[0]

In [ ]:
plans[1]

In [ ]:
kill_processes()